# Arcadia Fund: 360Giving Bulk Data

This represents the crawler for The Arcadia Fund's grants. 360giving has a [standard tabular format](https://standard.threesixtygiving.org/en/latest/#) for publishing UK philanthropies data. It is published as csv, xls, xlsx, depending on the funder.

### Notes:
- [Bulk Grant Reporting for Arcadia Fund](https://grantnav.threesixtygiving.org/org/360G-ArcadiaFund) includes funded grants both from Arcadia Fund and its sibling funder, [Lund Trust](https://lundtrust.org.uk/), a separate organization that funds environmental initiatives. For the sake of IOI's reporting (and until they get a valid ROR ID), we will only include grants from Arcadia Fund and will actively discard grants from the Lund Trust in this dataset.
- Arcadia Fund's [Grants Directory](https://www.arcadiafund.org.uk/grant-directory) has both web listings and 360giving bulk data. This tool will check for the latest version of the 360giving bulk data and use that as the source of truth.
- We **DO NOT** use the 360giving portal data for Arcadia Fund; its indexer has not updated since 2022.



In [30]:
from oic_scrape.items import AwardItem
import pandas as pd
from datetime import datetime
import polars as pl
from bs4 import BeautifulSoup
from attrs import asdict
import requests
from io import StringIO

## Parameters and Configuration

In [2]:
THREESIXTY_G_DATA_URL = (
    "https://www.arcadiafund.org.uk/uploads/360G-March-2024-DATA.csv"
)
USE_LATEST_BULK_DATA = True
FUNDER_ORG_NAME = "Arcadia Fund"
FUNDER_ORG_ROR_ID = "https://ror.org/051z6e826"
OUTPUT_LOCATION = "data/arcadia-fund--giving360_grants.jsonl"
OUTPUT_FORMAT = "jsonl"

In [3]:
def validate_output_format(format):
    """
    Validates the output file format.

    Args:
        format (str): The output format to be validated.

    Returns:
        bool: True if the format is valid (json, jsonl, or jsonlines), False otherwise.
    """
    if (
        format.lower() == "json"
        or format.lower() == "jsonl"
        or format.lower() == "jsonlines"
    ):
        return True
    else:
        return False


if validate_output_format(OUTPUT_FORMAT):
    if OUTPUT_FORMAT.lower() == "jsonl" or OUTPUT_FORMAT.lower() == "jsonlines":
        output_format_lines = True
    else:
        output_format_lines = False
else:
    raise ValueError("Output format should be either 'json' or 'jsonl'/'jsonlines'.")

In [35]:
def find_latest_results() -> str:
    """Finds the latest results link from the Arcadia Fund

    Returns:
        str: The URL of the latest results file
    """

    grants_directory_url = "https://www.arcadiafund.org.uk/grant-directory"

    response = requests.get(grants_directory_url)
    response.raise_for_status()  # Assuming 'response' is the HTML page response object
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the CSV link in the HTML page
    csv_link = None
    for link in soup.find_all("a"):
        href = link.get("href")
        if href.endswith(".csv"):
            csv_link = href
            break

    if csv_link:
        return csv_link
    else:
        raise ValueError(
            "No CSV link found in the Arcadia Fund's grant directory page."
        )


url = find_latest_results()
df = pd.read_csv(url)
crawl_ts = datetime.utcnow()
df["Award_Date_datetime"] = pd.to_datetime(
    df["Award Date"], format="%d/%m/%Y", errors="coerce"
)


In [36]:
awards = []
for ix, row in df.iterrows():

    # Process only Arcadia grants, not Lund Trust
    if row["Funding Org: Name"] == "ARCADIA":
        source = "arcadia.org.uk__360giving-export"
        grant_id = f"360g::{row['Identifier']}"
        funder_org_name = FUNDER_ORG_NAME
        recipient_org_name = row["Recipient Org:Name"]
        funder_org_ror_id = FUNDER_ORG_ROR_ID
        grant_year = int(row['Award year'])
        grant_start_date = row['Award_Date_datetime'].date()
        grant_duration = f"{row['Term (Months)']} months"
        award_amount = float(row['Amount Awarded'].replace('$', '').replace(',', '')) if row['Amount Awarded'] else None
        award_currency = row['Currency']
        award_amount_usd = award_amount
        grant_title = row["Title"]
        grant_description = row["Description"]
        program_of_funder = row["Grant Programme: Title"]
        program_of_funder += f" > {row['Grant Programme: Subtitle']}" if row["Grant Programme: Subtitle"] and row['Grant Programme: Subtitle'] != row['Grant Programme: Title'] else ""
        raw_source_data = row.to_json()

        award = AwardItem(
            _crawled_at=crawl_ts,
            source=source,
            grant_id=grant_id,
            funder_org_name=funder_org_name,
            funder_org_ror_id=FUNDER_ORG_ROR_ID,
            recipient_org_name=recipient_org_name,
            grant_year=grant_year,
            grant_start_date=grant_start_date,
            grant_duration=grant_duration,
            award_amount=award_amount if award_amount else None,
            award_currency=award_currency if award_currency else None,
            award_amount_usd=award_amount_usd if award_amount_usd else None,
            grant_title=grant_title,
            grant_description=str(grant_description),
            program_of_funder=program_of_funder,
            raw_source_data=raw_source_data,
        )
        awards.append(asdict(award))

In [44]:
pl.DataFrame(awards).write_ndjson(OUTPUT_LOCATION)